In [1]:
import pandas as pd
import os
os.getcwd()

'/home/niek/Elvina/Thesis/repository/Fact_Checking_model/explore'

In [2]:
actors_df = pd.read_csv('../data/raw/MovieSummaries/character.metadata.tsv', delimiter='\t', header=None)
column_names = ['Wikipedia ID','Freebase movie ID','Movie release date', 'Character name', 'Actor dob', 'Gender', 'Height','Ethnicity','Actor','Age', 'Freebase character/actor map ID','Freebase character ID', 'Freebase actor ID']
actors_df.columns = column_names
actors_df.drop(['Freebase movie ID','Movie release date', 'Character name', 'Actor dob', 'Gender', 'Height','Ethnicity','Age', 'Freebase character/actor map ID','Freebase character ID', 'Freebase actor ID'], axis=1, inplace=True)

print(actors_df)

        Wikipedia ID               Actor
0             975900      Wanda De Jesus
1             975900  Natasha Henstridge
2             975900            Ice Cube
3             975900       Jason Statham
4             975900         Clea DuVall
...              ...                 ...
450664        913762  Dorothy Elias-Fahn
450665        913762       Jonathan Fahn
450666      28308153      David Hemmings
450667      28308153    Roberta Paterson
450668      28308153         John Rogers

[450669 rows x 2 columns]


### A test to check if plot description is linked to the metadata via Wikipedia ID

In [3]:

# check if 'Wikipedia ID' column contains value 24225279
if (24225279 in actors_df['Wikipedia ID'].values):
    print("The value 24225279 is present in 'Wikipedia ID' column.")
else:
    print("The value 24225279 is not present in 'Wikipedia ID' column.")


The value 24225279 is present in 'Wikipedia ID' column.


## Metadata


In [4]:
data = pd.read_csv('../data/raw/MovieSummaries/movie.metadata.tsv', delimiter='\t', header=None)
column_names = ['Wikipedia ID','Freebase movie ID','Movie name', 'Year', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Genre']
data.columns = column_names
data.drop(['Freebase movie ID','Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries'], axis=1, inplace=True)

data['Genre'] = data['Genre'].apply(lambda x: eval(x).values())
data['Year'] = data['Year'].astype(str).str[:4]
data.loc[data['Year'] == '1010', 'Year'] = '2010'
#grouped_years = data.groupby('Year').size()
#grouped_years
df_combined = pd.merge(actors_df, data, on = 'Wikipedia ID')
df_combined.head(9)

def select_top_actors(group):
    # sort the group by actor's count and take the top 5
    top_actors = group.sort_values(by='Actor', ascending=False).head(5)
    return top_actors

# group the dataframe by the movie column and apply the select_top_actors function
df = df_combined.groupby('Movie name').apply(select_top_actors)

# reset the index to get rid of the groupby structure
df = df.reset_index(drop=True)

new_column_order = ['Wikipedia ID','Movie name', 'Year', 'Genre', 'Actor']
df = df.reindex(columns=new_column_order)



## Trying to make a list of actors instead of copying the same movies 5 times

In [5]:
# read the csv file
df = pd.read_csv('../data/processed/Movies_metadata.csv')

df['Actor'] = df['Actor'].astype(str)

# convert all columns to string type
df = df.astype(str)

# sort the DataFrame by "Wikipedia ID" and "Actor"
df = df.sort_values(['Wikipedia ID', 'Actor'])

# concatenate the "Actor" column for each movie using a comma
df = df.groupby(['Wikipedia ID', 'Movie name', 'Year', 'Genre'])['Actor'].apply(lambda x: ','.join(x)).reset_index()


print(df)


      Wikipedia ID                                 Movie name    Year  \
0         10000001                              Juan que reía  1976.0   
1         10000061  Los Muchachos de antes no usaban arsénico  1976.0   
2         10000067                          Brigada en acción  1977.0   
3         10000192                                    La Nona  1979.0   
4         10002175                               Our very own  2005.0   
...            ...                                        ...     ...   
62978      9998350                                La Película  1975.0   
62979      9998358                    Nazareno Cruz y el lobo  1975.0   
62980        99984                Una Aventura Llamada Menudo  1982.0   
62981      9999280                             Plaza de Almas     nan   
62982      9999915                           Alice to Nowhere     nan   

                                                   Genre  \
0                       dict_values(['Drama', 'Comedy'])   
1  

In [6]:
df = pd.read_csv('../data/processed/Movies_metadata.csv')

df1 = df.head(5)
df['Actor'] = df['Actor'].astype(str)
print(df1)

   Wikipedia ID           Movie name    Year  \
0      30332673  #1 Cheerleader Camp  2010.0   
1      30332673  #1 Cheerleader Camp  2010.0   
2      30332673  #1 Cheerleader Camp  2010.0   
3      30332673  #1 Cheerleader Camp  2010.0   
4      30332673  #1 Cheerleader Camp  2010.0   

                                               Genre                     Actor  
0  dict_values(['Sports', 'Sex comedy', 'Comedy f...  Starkesha Brown Robinson  
1  dict_values(['Sports', 'Sex comedy', 'Comedy f...              Sonja O'Hara  
2  dict_values(['Sports', 'Sex comedy', 'Comedy f...              Seth Cassell  
3  dict_values(['Sports', 'Sex comedy', 'Comedy f...               Seth Austin  
4  dict_values(['Sports', 'Sex comedy', 'Comedy f...    Sarah Kathryn Harrison  


In [7]:
actor_list = df1.groupby('Movie name').agg(lambda x: set(x)).reset_index()['Actor'].tolist()
print(actor_list)

[{'Seth Cassell', 'Seth Austin', 'Starkesha Brown Robinson', 'Sarah Kathryn Harrison', "Sonja O'Hara"}]


In [8]:
all_actors = list({actor for actors in actor_list for actor in actors})
print(all_actors)

['Seth Cassell', 'Seth Austin', 'Starkesha Brown Robinson', 'Sarah Kathryn Harrison', "Sonja O'Hara"]
